In [ ]:
import requests
import pandas as pd
from datetime import date, timedelta
import time
import base64
from IPython.display import HTML
from tqdm import tqdm


API_KEY = "RpaX4cXYXe5uxWJzOmZyxMwuj9k4CQlY"
TICKER = "NVDA"
ARTICLE_CAP_PER_DAY = 30


DATE_RANGES = [
    ("2022-09-01", "2023-09-01"),
    ("2023-09-02", "2024-09-01"),
    ("2024-09-02", "2025-09-01")
]

print(f" Starting 'Safe Mode' Fetch for {TICKER} (3-Year History)...")
print("   NOTE: We will sleep for 20 seconds between years to respect API limits.")

all_articles = []


for start_date, end_date in DATE_RANGES:
    print(f"\n📅 Fetching Batch: {start_date} to {end_date}...")

    base_url = "https://api.polygon.io/v2/reference/news"
    params = {
        "ticker": TICKER,
        "published_utc.gte": start_date,
        "published_utc.lte": end_date,
        "limit": 1000,
        "sort": "published_utc",
        "order": "desc",
        "apiKey": API_KEY
    }

    current_url = base_url
    batch_articles = []

    while True:
        if current_url == base_url:
            response = requests.get(current_url, params=params)
        else:
            response = requests.get(f"{current_url}&apiKey={API_KEY}")

        if response.status_code == 429:
            print("  Rate Limit Hit! Sleeping 60s...")
            time.sleep(60)
            continue

        data = response.json()

        if response.status_code == 200:
            results = data.get('results', [])
            batch_articles.extend(results)
            print(f"   -> Got {len(results)} articles (Batch Total: {len(batch_articles)})")

            if 'next_url' in data:
                current_url = data['next_url']
                time.sleep(0.5)
            else:
                break
        else:
            print(f" Error: {response.status_code}")
            break

    all_articles.extend(batch_articles)
    print(f" Finished Year. Total Articles so far: {len(all_articles)}")
    print(" Resting for 20 seconds to reset API quota...")
    time.sleep(20)


if len(all_articles) > 0:
    df = pd.DataFrame(all_articles)
    df['Date'] = pd.to_datetime(df['published_utc']).dt.date


    df_capped = df.groupby('Date').head(ARTICLE_CAP_PER_DAY).copy()


    df_capped['Source'] = df_capped['publisher'].apply(lambda x: x.get('name') if isinstance(x, dict) else 'Unknown')
    df_clean = df_capped[['Date', 'title', 'article_url', 'Source', 'description']].rename(columns={
        'title': 'Headline', 'article_url': 'Link', 'description': 'Summary'
    })


    full_date_range = pd.date_range(start="2022-09-01", end="2025-09-01").date
    df_dates = pd.DataFrame(full_date_range, columns=['Date'])

    df_final = pd.merge(df_dates, df_clean, on='Date', how='left')
    df_final['Headline'] = df_final['Headline'].fillna("No News")
    df_final['Summary'] = df_final['Summary'].fillna("Market Closed or Quiet Day")
    df_final['Source'] = df_final['Source'].fillna("None")


    filename = f"{TICKER}_Sept_2022_to_Sept_2025_News_COMPLETE.csv"
    df_final.to_csv(filename, index=False)

    print("\n" + "="*50)
    print(f"🎉 SUCCESS! Downloaded {len(df_final)} rows.")
    print("="*50)


    with open(filename, 'rb') as f:
        b64 = base64.b64encode(f.read()).decode()
    payload = f'data:text/csv;base64,{b64}'
    html = f'<a download="{filename}" href="{payload}" target="_blank"><button style="background-color:#4CAF50;color:white;padding:10px;">DOWNLOAD COMPLETE 3-YEAR CSV</button></a>'
    display(HTML(html))
else:
    print(" Failed to fetch any articles.")

In [ ]:

# STAGE 2: GPU-OPTIMIZED
!pip install transformers torch pandas tqdm datasets

import pandas as pd
import torch
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset
from tqdm.auto import tqdm
import os

INPUT_FILE = "NVDA_Sept_2022_to_Sept_2025_News_COMPLETE.csv"
OUTPUT_FILE = "NVDA_3Year_Processed_Data.csv"
BATCH_SIZE = 64

device = 0 if torch.cuda.is_available() else -1
print(f" Hardware Check: {'GPU ONLINE' if device==0 else ' CPU ONLY'}")

if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE)
    df = df.dropna(subset=['Headline', 'Summary'])

    df['Headline'] = df['Headline'].astype(str)
    df['Summary'] = df['Summary'].astype(str)

    print(f" Loaded {len(df)} articles.")
else:
    print(f" Error: '{INPUT_FILE}' not found.")
    df = pd.DataFrame()

if not df.empty:
    print("\n Loading AI Models...")

    df['AI_Text'] = df['Headline'] + ". " + df['Summary']

    hf_dataset = Dataset.from_pandas(df[['AI_Text']])

    sentiment_pipe = pipeline(
        "text-classification",
        model="ProsusAI/finbert",
        device=device,
        batch_size=BATCH_SIZE,
        truncation=True,
        max_length=512
    )

    relation_pipe = pipeline(
        "zero-shot-classification",
        model="valhalla/distilbart-mnli-12-3",
        device=device,
        batch_size=BATCH_SIZE
    )
    RELATION_LABELS = ["Competitor", "Supplier", "Customer", "Partner", "Regulatory", "Neutral"]

    print(f"\n⚡ Streaming {len(df)} articles to GPU...")

    print("   ... Calculating Sentiment")
    sent_scores = []

    for out in tqdm(sentiment_pipe(KeyDataset(hf_dataset, "AI_Text")), total=len(df)):
        score = out['score'] if out['label'] == 'positive' else -out['score'] if out['label'] == 'negative' else 0.0
        sent_scores.append(score)

    print("   ... Calculating Relationships")
    rel_types = []

    for out in tqdm(relation_pipe(KeyDataset(hf_dataset, "AI_Text"), candidate_labels=RELATION_LABELS), total=len(df)):
        rel_types.append(out['labels'][0])

    df['Sentiment_Score'] = sent_scores
    df['Relation_Type'] = rel_types

    df_final = df[['Date', 'Headline', 'Sentiment_Score', 'Relation_Type']]
    df_final.to_csv(OUTPUT_FILE, index=False)

    print(f"\n SUCCESS! Processed 3 Years of Data.")
    print(f"   Saved to: {OUTPUT_FILE}")

In [ ]:
import torch.nn.functional as F
import yfinance as yf

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler

print("Imports successful!")

In [ ]:

# RAW DATA -> TRADING MODEL, BUILDS the Knowledge Graph

!pip install torch_geometric

import torch.nn.functional as F
import yfinance as yf
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import yfinance as yf
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import random
from tqdm.auto import tqdm


print("🔍 Searching for data...")

possible_files = [
    "NVDA_Sept_2022_to_Sept_2025_News_COMPLETE.csv",
    "NVDA_Sept_2022_to_Sept_2025_News.csv",
    "NVDA_3Year_Processed_Data.csv"
]

found_file = None
for f in possible_files:
    if os.path.exists(f):
        found_file = f
        break

if not found_file:
    raise FileNotFoundError(" No CSV found! Please run your 'Fetch' code again to generate the news file.")

print(f"Found File: {found_file}")
df = pd.read_csv(found_file)

if 'Sentiment_Score' not in df.columns:
    print(" Missing AI Columns. Generating them NOW (Fast Mode)...")

    def fast_sentiment(text):
        text = str(text).lower()
        if any(w in text for w in ['soar', 'record', 'jump', 'beat', 'bull', 'buy', 'partner']): return 0.9
        if any(w in text for w in ['drop', 'miss', 'fail', 'ban', 'sanction', 'sell', 'lawsuit']): return -0.9
        return 0.1

    df['Sentiment_Score'] = df['Headline'].apply(fast_sentiment)

    def fast_relation(text):
        text = str(text).lower()
        if 'amd' in text or 'intel' in text: return 'Competitor'
        if 'tsmc' in text or 'supply' in text: return 'Supplier'
        if 'openai' in text or 'microsoft' in text: return 'Partner'
        return 'General_Market'

    df['Relation_Type'] = df['Headline'].apply(fast_relation)
    print(" AI Columns Generated.")

df.rename(columns={'Sentiment_Score': 'Weight', 'Relation_Type': 'Target'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

print(f"🕸️ Building Dynamic Graph (RAM)...")
temporal_graphs = {}
unique_dates = sorted(df['Date'].unique())
DECAY_RATE = 0.95

for current_date in tqdm(unique_dates, desc="Processing"):

    cutoff = current_date - pd.Timedelta(days=60)
    history = df[(df['Date'] <= current_date) & (df['Date'] > cutoff)].copy()

    history['Days_Old'] = (current_date - history['Date']).dt.days
    history['Decayed_Weight'] = history['Weight'] * (DECAY_RATE ** history['Days_Old'])

    G = nx.MultiDiGraph()
    for _, row in history.iterrows():
        if abs(row['Decayed_Weight']) > 0.05:
            G.add_edge("Nvidia", row['Target'], weight=row['Decayed_Weight'])

    temporal_graphs[current_date.strftime('%Y-%m-%d')] = G

print("📉 Fetching Prices...")
graph_dates = sorted(list(temporal_graphs.keys()))
start, end = graph_dates[0], graph_dates[-1]
fetch_start = (pd.to_datetime(start) - pd.Timedelta(days=60)).strftime('%Y-%m-%d')

df_price = yf.download("NVDA", start=fetch_start, end=end, progress=False)
if isinstance(df_price.columns, pd.MultiIndex): df_price.columns = [c[0] for c in df_price.columns]
df_price.reset_index(inplace=True)
df_price['Date'] = df_price['Date'].dt.strftime('%Y-%m-%d')

df_price['SMA'] = df_price['Close'].rolling(14).mean()
df_price['Return'] = df_price['Close'].pct_change()
df_price['Vol'] = df_price['Close'].rolling(14).std()
df_price.dropna(inplace=True)

scaler = MinMaxScaler()
kpi_cols = ['Close', 'SMA', 'Return', 'Vol']
scaled_data = scaler.fit_transform(df_price[kpi_cols].values)
date_to_kpi = {r['Date']: scaled_data[i] for i, r in df_price.iterrows()}

all_nodes = set()
for g in temporal_graphs.values(): all_nodes.update(g.nodes())
node_to_idx = {n: i for i, n in enumerate(sorted(list(all_nodes)))}
NUM_NODES = len(node_to_idx)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_pyg(date_str):
    g = temporal_graphs.get(date_str)
    if not g or g.number_of_edges() == 0: return None
    edges = [[node_to_idx[u], node_to_idx[v]] for u,v in g.edges()]
    weights = [d['weight'] for u,v,d in g.edges(data=True)]
    return Data(x=torch.eye(NUM_NODES), edge_index=torch.tensor(edges).t().contiguous(), edge_attr=torch.tensor(weights).float())

seqs, targets, target_dates = [], [], []
valid_dates = sorted([d for d in graph_dates if d in date_to_kpi])
SEQ_LEN = 5

for i in range(len(valid_dates) - SEQ_LEN):
    dates_win = valid_dates[i:i+SEQ_LEN]
    target_d = valid_dates[i+SEQ_LEN]
    gs = [get_pyg(d) for d in dates_win]
    if any(g is None for g in gs): continue
    seqs.append((gs, [date_to_kpi[d] for d in dates_win]))
    targets.append(date_to_kpi[target_d][0])
    target_dates.append(target_d)

train_size = len(seqs) - 60
train_data = seqs[:train_size]
test_data = seqs[train_size:]
train_y = targets[:train_size]
test_y = targets[train_size:]
test_dates = target_dates[train_size:]

class FastTrader(nn.Module):
    def __init__(self):
        super().__init__()
        self.gnn = GCNConv(NUM_NODES, 32)
        self.lstm = nn.LSTM(32+4, 64, batch_first=True)
        self.head = nn.Linear(64, 1)

    def forward(self, gs, kpis):
        embeds = []
        for g in gs:
            x, idx, w = g.x.to(device), g.edge_index.to(device), g.edge_attr.to(device)
            x = torch.tanh(self.gnn(x, idx, w))
            embeds.append(global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long).to(device)))
        return self.head(self.lstm(torch.cat((torch.stack(embeds, 1), kpis), 2))[0][:, -1, :])

print(f"\n🚀 Training on {device}...")
model = FastTrader().to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

for ep in range(30):
    model.train()
    tot_loss = 0
    for i in range(len(train_data)):
        gs, kpis = train_data[i]
        tgt = torch.tensor([[train_y[i]]]).float().to(device)
        kpi_in = torch.tensor(kpis).float().unsqueeze(0).to(device)

        opt.zero_grad()
        loss = loss_fn(model(gs, kpi_in), tgt)
        loss.backward()
        opt.step()
        tot_loss += loss.item()
    if ep % 5 == 0: print(f"Epoch {ep} | Loss: {tot_loss/len(train_data):.5f}")

model.eval()
preds, acts = [], []
for i in range(len(test_data)):
    gs, kpis = test_data[i]
    pred = model(gs, torch.tensor(kpis).float().unsqueeze(0).to(device)).item()

    d = np.zeros((1, 4))
    d[0,0] = pred
    preds.append(scaler.inverse_transform(d)[0,0])
    d[0,0] = test_y[i]
    acts.append(scaler.inverse_transform(d)[0,0])

y_act, y_pred = np.array(acts), np.array(preds)
acc = accuracy_score((np.diff(y_act) > 0), ((y_pred[1:] - y_act[:-1]) > 0)) * 100
print(f"\n FINAL ACCURACY: {acc:.2f}%")

plt.figure(figsize=(10, 5))
plt.plot(test_dates, acts, label='Actual')
plt.plot(test_dates, preds, label='AI Prediction', linestyle='--')
plt.title(f"Fixer Script Result (Acc: {acc:.2f}%)"); plt.legend(); plt.show()

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import pickle

input_file = 'NVDA_3Year_Processed_Data.csv'
output_file = 'NVDA_Dynamic_Graph.pkl'

relation_types = ['Ego', 'Competitor', 'Regulatory', 'Partner', 'Supplier', 'Customer', 'Neutral']
rel_to_idx = {r: i for i, r in enumerate(relation_types)}
num_relations = len(relation_types)

def create_knowledge_graph(csv_path, pkl_path):
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)

    grouped = df.groupby('Date')

    dataset = []

    print("Processing graphs...")
    for date, group in grouped:

        num_news = len(group)
        num_nodes = 1 + num_news

        x = torch.zeros((num_nodes, num_relations + 1), dtype=torch.float)

        x[0, rel_to_idx['Ego']] = 1.0
        x[0, -1] = 0.0

        for i, (_, row) in enumerate(group.iterrows()):
            node_idx = i + 1
            rel_type = row['Relation_Type']
            sentiment = row['Sentiment_Score']

            if rel_type in rel_to_idx:
                x[node_idx, rel_to_idx[rel_type]] = 1.0

            x[node_idx, -1] = sentiment

        sources = torch.arange(1, num_nodes, dtype=torch.long)
        targets = torch.zeros(num_news, dtype=torch.long)

        edge_index = torch.stack([sources, targets], dim=0)

        data = Data(x=x, edge_index=edge_index)

        data.date = date
        data.num_nodes = num_nodes

        dataset.append(data)

    print(f"Saving {len(dataset)} graphs to {pkl_path}...")
    with open(pkl_path, 'wb') as f:
        pickle.dump(dataset, f)

    print("Done!")

if __name__ == "__main__":
    create_knowledge_graph(input_file, output_file)

In [ ]:
import pickle
import torch
from torch_geometric.data import Data

with open('NVDA_Dynamic_Graph.pkl', 'rb') as f:
    graph_dicts = pickle.load(f)

dataset = []
for g in graph_dicts:
    x = torch.tensor(g['x'], dtype=torch.float)
    edge_index = torch.tensor(g['edge_index'], dtype=torch.long)

    data = Data(x=x, edge_index=edge_index)

    data.date = g['date']

    dataset.append(data)

print(f"Successfully loaded {len(dataset)} daily graph snapshots.")
print(dataset[0])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import pickle


# Define the GCN Model

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(num_node_features, hidden_channels)

        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):

        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

print("Loading graph data...")
with open('NVDA_Dynamic_Graph.pkl', 'rb') as f:
    dataset = pickle.load(f)

sample_graph = dataset[0]
num_node_features = sample_graph.x.shape[1]

print(f"Loaded {len(dataset)} graphs.")
print(f"Feature size per node: {num_node_features}")

HIDDEN_CHANNELS = 16
NUM_CLASSES = 2

model = GCN(num_node_features, HIDDEN_CHANNELS, NUM_CLASSES)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
sample_graph = sample_graph.to(device)

print("\nModel Architecture:")
print(model)

model.eval()
with torch.no_grad():
    out = model(sample_graph)

print("\nOutput shape from first graph:", out.shape)
print("First 5 node predictions (Log Softmax):\n", out[:5])

In [ ]:

# GCN-LSTM: YFINANCE + STRICT DATA

try:
    import torch_geometric
except ImportError:
    # !pip install torch_geometric
    import torch_geometric

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import pickle
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random
import os

TICKER = "NVDA"

START_DATE = "2022-09-01"
END_DATE = "2025-09-02"
TRAIN_CUTOFF = "2025-06-15"

SEED = 42
SEQ_LEN = 5
EPOCHS = 50
HIDDEN_DIM = 128

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Training on: {device}")


print("\n1️⃣ Loading Knowledge Graph...")
try:
    with open('NVDA_Dynamic_Graph.pkl', 'rb') as f:
        raw_data = pickle.load(f)
except FileNotFoundError:
    print(" Error: 'NVDA_Dynamic_Graph.pkl' not found.")
    exit()

graph_map = {}
for item in raw_data:
    x = torch.tensor(item['x'], dtype=torch.float)
    edge_index = torch.tensor(item['edge_index'], dtype=torch.long)
    data = Data(x=x, edge_index=edge_index)
    d_str = pd.to_datetime(item['date']).strftime('%Y-%m-%d')
    graph_map[d_str] = data

print(f" Loaded {len(graph_map)} graph snapshots.")


print(f"\n2️ Fetching Stock Data via yfinance ({START_DATE} -> {END_DATE})...")

df = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [c[0] for c in df.columns]

df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

print(f" Fetched {len(df)} rows.")

df['SMA_14'] = df['Close'].rolling(window=14).mean()
df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
df['Return'] = df['Close'].pct_change()

delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))


df['Target_Price'] = df['Close'].shift(-1)
df.dropna(inplace=True)


print(f"\n3️ Splitting Data (Cutoff: {TRAIN_CUTOFF})...")

train_df = df[df['Date'] <= TRAIN_CUTOFF].copy()
test_df  = df[df['Date'] > TRAIN_CUTOFF].copy()

print(f"   Train Rows: {len(train_df)}")
print(f"   Test Rows:  {len(test_df)}")

KPI_COLS = ['Close', 'RSI', 'MACD', 'SMA_14', 'Return', 'Target_Price']
scaler = MinMaxScaler()
scaler.fit(train_df[KPI_COLS])

train_df[KPI_COLS] = scaler.transform(train_df[KPI_COLS])
test_df[KPI_COLS]  = scaler.transform(test_df[KPI_COLS])

full_df_scaled = pd.concat([train_df, test_df])
date_to_features = {row['Date']: row[KPI_COLS[:-1]].values.astype(float) for i, row in full_df_scaled.iterrows()}
date_to_target   = {row['Date']: row['Target_Price'] for i, row in full_df_scaled.iterrows()}


sequences = []
targets = []
target_dates = []

valid_stock_dates = set(date_to_features.keys())
valid_graph_dates = set(graph_map.keys())
common_dates = sorted(list(valid_stock_dates.intersection(valid_graph_dates)))

print(f"   Aligned {len(common_dates)} common days.")

for i in range(len(common_dates) - SEQ_LEN):
    seq_dates = common_dates[i : i+SEQ_LEN]
    target_date = seq_dates[-1]

    if target_date not in date_to_target: continue

    seq_graphs = [graph_map[d] for d in seq_dates]
    seq_kpis = [date_to_features[d] for d in seq_dates]
    target_val = date_to_target[target_date]

    sequences.append((seq_graphs, seq_kpis))
    targets.append(target_val)
    target_dates.append(target_date)

train_seqs, train_targets = [], []
test_seqs, test_targets, test_dates_list = [], [], []

for i, date in enumerate(target_dates):
    if date <= TRAIN_CUTOFF:
        train_seqs.append(sequences[i])
        train_targets.append(targets[i])
    else:
        test_seqs.append(sequences[i])
        test_targets.append(targets[i])
        test_dates_list.append(date)

print(f"   Final Train Seqs: {len(train_seqs)}")
print(f"   Final Test Seqs:  {len(test_seqs)}")

# MODEL (GCN + LSTM)
class IntegratedTrader(nn.Module):
    def __init__(self, node_feat_dim, kpi_dim, gnn_out=32, lstm_hidden=128):
        super(IntegratedTrader, self).__init__()
        self.gnn1 = GCNConv(node_feat_dim, 64)
        self.gnn2 = GCNConv(64, gnn_out)
        self.lstm = nn.LSTM(gnn_out + kpi_dim, lstm_hidden, batch_first=True, dropout=0.2)
        self.head = nn.Linear(lstm_hidden, 1)

    def forward(self, graph_list, kpi_tensor):
        sentiment_vecs = []
        for g in graph_list:
            x, edge_index = g.x.to(device), g.edge_index.to(device)
            x = F.relu(self.gnn1(x, edge_index))
            x = self.gnn2(x, edge_index)
            batch = torch.zeros(x.size(0), dtype=torch.long).to(device)
            day_vec = global_mean_pool(x, batch)
            sentiment_vecs.append(day_vec)

        sentiment_seq = torch.stack(sentiment_vecs, dim=1)
        fusion = torch.cat((sentiment_seq, kpi_tensor), dim=2)
        lstm_out, _ = self.lstm(fusion)
        return self.head(lstm_out[:, -1, :])

class AsymmetricDirectionalLoss(nn.Module):
    def __init__(self, penalty=2.0, bullish_weight=4.0):
        super().__init__()
        self.mse = nn.MSELoss()
        self.penalty = penalty
        self.bullish_weight = bullish_weight

    def forward(self, pred, target, prev_price):
        loss_mse = self.mse(pred, target)
        true_diff = target - prev_price
        pred_diff = pred - prev_price
        wrong_dir = (true_diff * pred_diff) < 0
        missed_up = (true_diff > 0) & wrong_dir
        dir_cost = torch.abs(true_diff - pred_diff)
        weighted_cost = torch.where(missed_up, dir_cost * self.bullish_weight, dir_cost)
        return loss_mse + (self.penalty * torch.mean(wrong_dir.float() * weighted_cost))

if not train_seqs:
    print(" Error: No training sequences found.")
    exit()

sample_graph = sequences[0][0][0]
NODE_DIM = sample_graph.x.shape[1]
KPI_DIM = len(KPI_COLS) - 1

model = IntegratedTrader(NODE_DIM, KPI_DIM, lstm_hidden=HIDDEN_DIM).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)
criterion = AsymmetricDirectionalLoss(penalty=2.0, bullish_weight=4.0)

print(f"\n🚀 Starting Training ({EPOCHS} Epochs)...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for i in range(len(train_seqs)):
        seq_graphs, seq_kpis = train_seqs[i]
        target = torch.tensor([[train_targets[i]]], dtype=torch.float).to(device)
        kpi_input = torch.tensor(np.array(seq_kpis), dtype=torch.float).unsqueeze(0).to(device)
        prev_price = kpi_input[:, -1, 0].unsqueeze(1)

        optimizer.zero_grad()
        pred = model(seq_graphs, kpi_input)
        loss = criterion(pred, target, prev_price)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch+1) % 5 == 0:
        print(f"   Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss/len(train_seqs):.5f}")

MODEL_FILE = "NVDA_Sniper_Model.pth"
torch.save(model.state_dict(), MODEL_FILE)
print(f"\n Model saved to: {MODEL_FILE}")
print("You can now run the Live Agent script!")

print("\n" + "="*50)
print(f" FINAL EVALUATION (16 June 2025 -> 1 Sept 2025)")
print("="*50)

model.eval()
preds, actuals = [], []

with torch.no_grad():
    for i in range(len(test_seqs)):
        seq_graphs, seq_kpis = test_seqs[i]
        kpi_input = torch.tensor(np.array(seq_kpis), dtype=torch.float).unsqueeze(0).to(device)
        pred = model(seq_graphs, kpi_input).item()
        preds.append(pred)
        actuals.append(test_targets[i])

def unscale(vals):
    dummy = np.zeros((len(vals), len(KPI_COLS)))
    dummy[:, -1] = vals
    return scaler.inverse_transform(dummy)[:, -1]

final_preds = unscale(preds)
final_acts = unscale(actuals)

prev_prices_scaled = [seq[1][-1][0] for seq in test_seqs]
dummy_prev = np.zeros((len(prev_prices_scaled), len(KPI_COLS)))
dummy_prev[:, 0] = prev_prices_scaled
prev_prices_real = scaler.inverse_transform(dummy_prev)[:, 0]

mae = mean_absolute_error(final_acts, final_preds)
true_dir = (final_acts > prev_prices_real).astype(int)
pred_dir = (final_preds > prev_prices_real).astype(int)

acc = accuracy_score(true_dir, pred_dir)
prec = precision_score(true_dir, pred_dir, zero_division=0)
rec = recall_score(true_dir, pred_dir, zero_division=0)
f1 = f1_score(true_dir, pred_dir, zero_division=0)

print(f" MAE: ${mae:.2f}")
print(f" Accuracy:  {acc:.2%}")
print(f" Precision: {prec:.2%}")
print(f" Recall:    {rec:.2%}")
print(f" F1 Score:  {f1:.4f}")


plot_dates = [pd.to_datetime(d) for d in test_dates_list]

correct_indices = [i for i in range(len(true_dir)) if true_dir[i] == pred_dir[i]]
incorrect_indices = [i for i in range(len(true_dir)) if true_dir[i] != pred_dir[i]]

plt.figure(figsize=(14, 7))
plt.plot(plot_dates, final_acts, label='Actual Price', color='black', alpha=0.6)
plt.plot(plot_dates, final_preds, label='AI Prediction', color='blue', linestyle='--')

plt.scatter([plot_dates[i] for i in correct_indices], [final_acts[i] for i in correct_indices],
            color='green', marker='^', s=50, label='Correct')
plt.scatter([plot_dates[i] for i in incorrect_indices], [final_acts[i] for i in incorrect_indices],
            color='red', marker='v', s=50, label='Wrong')

plt.title(f'NVDA Prediction (YFinance) | Acc: {acc:.2%}', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
# Run this NOW to save the model currently in your RAM
torch.save(model.state_dict(), "NVDA_Sniper_Model.pth")
print(" Saved! You can now run the Agent.")

In [ ]:

# NVDA TIME-TRAVEL TRADING AGENT

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import yfinance as yf
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

TICKER = "NVDA"
MODEL_FILE = "NVDA_Sniper_Model.pth"
SEQ_LEN = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_trading_day_before(target_date_str):
    """Finds the trading day immediately BEFORE the target date."""
    target_dt = datetime.strptime(target_date_str, '%Y-%m-%d')

    prev_dt = target_dt - timedelta(days=1)
    while prev_dt.weekday() > 4:
        prev_dt -= timedelta(days=1)

    return prev_dt.strftime('%Y-%m-%d')

def get_historical_stock_data(end_date_str):
    """
    Fetches stock data ending EXACTLY on end_date_str.
    We need the last SEQ_LEN days leading up to this date.
    """
    print(f" Fetching Stock Data up to {end_date_str}...")

    end_dt_obj = datetime.strptime(end_date_str, '%Y-%m-%d') + timedelta(days=1)
    query_end = end_dt_obj.strftime('%Y-%m-%d')
    start_dt_obj = end_dt_obj - timedelta(days=90)
    query_start = start_dt_obj.strftime('%Y-%m-%d')

    df = yf.download(TICKER, start=query_start, end=query_end, progress=False)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]

    df.reset_index(inplace=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    df['SMA_14'] = df['Close'].rolling(window=14).mean()
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Return'] = df['Close'].pct_change()

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    df_filtered = df[df['Date'] <= end_date_str].copy()

    return df_filtered.tail(SEQ_LEN)

def update_knowledge_graph(date_str):
    """
    Simulates fetching news for 'date_str' and updating the graph.
    In a real app, this would scrape news and decay old weights.
    """
    print(f"🕸️ Updating Knowledge Graph for {date_str}...")

    num_nodes = 50
    x = torch.eye(num_nodes)
    edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)

    return Data(x=x, edge_index=edge_index)

class IntegratedTrader(nn.Module):
    def __init__(self, node_feat_dim, kpi_dim, gnn_out=32, lstm_hidden=128):
        super(IntegratedTrader, self).__init__()
        self.gnn1 = GCNConv(node_feat_dim, 64)
        self.gnn2 = GCNConv(64, gnn_out)
        self.lstm = nn.LSTM(gnn_out + kpi_dim, lstm_hidden, batch_first=True, dropout=0.2)
        self.head = nn.Linear(lstm_hidden, 1)

    def forward(self, graph_list, kpi_tensor):
        sentiment_vecs = []
        for g in graph_list:
            x, edge_index = g.x.to(DEVICE), g.edge_index.to(DEVICE)
            x = F.relu(self.gnn1(x, edge_index))
            x = self.gnn2(x, edge_index)
            batch = torch.zeros(x.size(0), dtype=torch.long).to(DEVICE)
            day_vec = global_mean_pool(x, batch)
            sentiment_vecs.append(day_vec)

        sentiment_seq = torch.stack(sentiment_vecs, dim=1)
        fusion = torch.cat((sentiment_seq, kpi_tensor), dim=2)
        lstm_out, _ = self.lstm(fusion)
        return self.head(lstm_out[:, -1, :])

def run_prediction(target_date_str):
    print("\n" + "="*40)
    print(f" AGENT ACTIVATED | TARGET: {target_date_str}")
    print("="*40)

    decision_date_str = get_trading_day_before(target_date_str)
    print(f" Decision Context Date: {decision_date_str}")

    df_seq = get_historical_stock_data(decision_date_str)

    if len(df_seq) < SEQ_LEN:
        print(f" Error: Not enough data found ending on {decision_date_str}")
        print("   (Check if the date is a valid trading day or too far in the past/future)")
        return

    KPI_COLS = ['Close', 'RSI', 'MACD', 'SMA_14', 'Return']
    scaler = MinMaxScaler()
    df_scaled = df_seq.copy()

    df_scaled[KPI_COLS] = scaler.fit_transform(df_seq[KPI_COLS])

    seq_kpis = df_scaled[KPI_COLS].values
    kpi_tensor = torch.tensor(seq_kpis, dtype=torch.float).unsqueeze(0).to(DEVICE)

    graph_snapshot = update_knowledge_graph(decision_date_str)
    graph_seq = [graph_snapshot] * SEQ_LEN

    NODE_DIM = 50
    KPI_DIM = len(KPI_COLS)

    model = IntegratedTrader(node_feat_dim=NODE_DIM, kpi_dim=KPI_DIM, lstm_hidden=128).to(DEVICE)

    if os.path.exists(MODEL_FILE):
        try:
            model.load_state_dict(torch.load(MODEL_FILE, map_location=DEVICE))
            print(f" Loaded Trained Model: {MODEL_FILE}")
            model.eval()

            with torch.no_grad():
                pred_scaled = model(graph_seq, kpi_tensor).item()
        except Exception as e:
            print(f" Error loading model: {e}")
            pred_scaled = 0.5
    else:
        print("\n WARNING: 'NVDA_Sniper_Model.pth' NOT FOUND.")
        print("   >> Running with UNTRAINED (Random) weights to demonstrate logic.")
        print("   >> The prediction below is technically meaningless until you train/save the model.")
        model.eval()
        with torch.no_grad():
            pred_scaled = model(graph_seq, kpi_tensor).item()

    dummy = np.zeros((1, len(KPI_COLS)))
    dummy[0, 0] = pred_scaled
    price_pred = scaler.inverse_transform(dummy)[0, 0]

    last_close = df_seq['Close'].iloc[-1]

    print("\n" + "-"*30)
    print(f" Close on {decision_date_str}:  ${last_close:.2f}")
    print(f" Forecast for {target_date_str}:  ${price_pred:.2f}")

    change_pct = ((price_pred - last_close) / last_close) * 100
    print(f"📊 Predicted Move: {change_pct:.2f}%")

    if price_pred > last_close * 1.005:
        print(" SIGNAL: BUY (Bullish)")
    elif price_pred < last_close * 0.995:
        print(" SIGNAL: SELL (Bearish)")
    else:
        print(" SIGNAL: HOLD (Neutral)")
    print("-"*30)

if __name__ == "__main__":
    target = "2025-09-05"
    run_prediction(target)

In [ ]:
pip install streamlit plotly

In [ ]:
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
import plotly.graph_objects as go
import os

TICKER = "NVDA"
MODEL_FILE = "NVDA_Sniper_Model.pth"
SEQ_LEN = 5
DEVICE = torch.device('cpu')

class IntegratedTrader(nn.Module):
    def __init__(self, node_feat_dim, kpi_dim, gnn_out=32, lstm_hidden=128):
        super(IntegratedTrader, self).__init__()
        self.gnn1 = GCNConv(node_feat_dim, 64)
        self.gnn2 = GCNConv(64, gnn_out)
        self.lstm = nn.LSTM(gnn_out + kpi_dim, lstm_hidden, batch_first=True, dropout=0.2)
        self.head = nn.Linear(lstm_hidden, 1)

    def forward(self, graph_list, kpi_tensor):
        sentiment_vecs = []
        for g in graph_list:
            x, edge_index = g.x.to(DEVICE), g.edge_index.to(DEVICE)
            x = F.relu(self.gnn1(x, edge_index))
            x = self.gnn2(x, edge_index)
            batch = torch.zeros(x.size(0), dtype=torch.long).to(DEVICE)
            day_vec = global_mean_pool(x, batch)
            sentiment_vecs.append(day_vec)

        sentiment_seq = torch.stack(sentiment_vecs, dim=1)
        fusion = torch.cat((sentiment_seq, kpi_tensor), dim=2)
        lstm_out, _ = self.lstm(fusion)
        return self.head(lstm_out[:, -1, :])

def get_historical_stock_data(end_date_str):
    """Fetches data ending exactly on the decision day."""
    end_dt_obj = datetime.strptime(end_date_str, '%Y-%m-%d') + timedelta(days=1)
    query_end = end_dt_obj.strftime('%Y-%m-%d')
    start_dt_obj = end_dt_obj - timedelta(days=90)
    query_start = start_dt_obj.strftime('%Y-%m-%d')

    df = yf.download(TICKER, start=query_start, end=query_end, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    # Indicators
    df['SMA_14'] = df['Close'].rolling(window=14).mean()
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Return'] = df['Close'].pct_change()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    return df[df['Date'] <= end_date_str].copy()

def get_trading_day_before(target_date_obj):
    """Finds previous trading day."""
    prev_dt = target_date_obj - timedelta(days=1)
    while prev_dt.weekday() > 4:
        prev_dt -= timedelta(days=1)
    return prev_dt.strftime('%Y-%m-%d')


st.set_page_config(page_title="NVDA Sniper Agent", page_icon="🎯", layout="wide")

st.title("🎯 NVDA Sniper: AI Investment Agent")
st.markdown("""
**Wanna invest in NVIDIA?** Let the AI check the **Knowledge Graph** and **Price Trends** for you.
""")

st.sidebar.header("⚙️ Configuration")
target_date_input = st.sidebar.date_input("Target Date for Prediction", datetime.today() + timedelta(days=1))
target_date_str = target_date_input.strftime('%Y-%m-%d')

if st.sidebar.button("🚀 Run Analysis"):
    with st.spinner(f"Traveling back in time to analyze context for {target_date_str}..."):

        decision_date_str = get_trading_day_before(target_date_input)

        df_seq = get_historical_stock_data(decision_date_str)

        if len(df_seq) < SEQ_LEN:
            st.error(f"Not enough data found ending on {decision_date_str}. Try a more recent date.")
        else:

            KPI_COLS = ['Close', 'RSI', 'MACD', 'SMA_14', 'Return']
            scaler = MinMaxScaler()
            df_scaled = df_seq.copy()
            df_scaled[KPI_COLS] = scaler.fit_transform(df_seq[KPI_COLS])

            seq_kpis = df_scaled[KPI_COLS].tail(SEQ_LEN).values
            kpi_tensor = torch.tensor(seq_kpis, dtype=torch.float).unsqueeze(0).to(DEVICE)

            x = torch.eye(50)
            edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
            graph_snapshot = Data(x=x, edge_index=edge_index)
            graph_seq = [graph_snapshot] * SEQ_LEN

            model = IntegratedTrader(node_feat_dim=50, kpi_dim=len(KPI_COLS), lstm_hidden=128).to(DEVICE)

            model_loaded = False
            if os.path.exists(MODEL_FILE):
                try:
                    model.load_state_dict(torch.load(MODEL_FILE, map_location=DEVICE))
                    model.eval()
                    model_loaded = True
                except:
                    st.warning(" Model file corrupted. Using Random Weights (Demo Mode).")
            else:
                st.warning(" 'NVDA_Sniper_Model.pth' not found. Using Random Weights (Demo Mode).")

            with torch.no_grad():
                pred_scaled = model(graph_seq, kpi_tensor).item() if model_loaded else 0.55

            dummy = np.zeros((1, len(KPI_COLS)))
            dummy[0, 0] = pred_scaled
            price_pred = scaler.inverse_transform(dummy)[0, 0]

            last_close = df_seq['Close'].iloc[-1]
            change_pct = ((price_pred - last_close) / last_close) * 100

            col1, col2, col3 = st.columns(3)
            col1.metric("Last Close Price", f"${last_close:.2f}", f"On {decision_date_str}")
            col2.metric("AI Target Price", f"${price_pred:.2f}", f"{change_pct:.2f}%")

            if price_pred > last_close * 1.005:
                signal = "BUY "
                color = "green"
            elif price_pred < last_close * 0.995:
                signal = "SELL "
                color = "red"
            else:
                signal = "HOLD "
                color = "gray"

            col3.markdown(f"### Signal: :{color}[{signal}]")

            st.markdown("###  Price Movement & Prediction")

            fig = go.Figure()

            plot_df = df_seq.tail(30)
            fig.add_trace(go.Scatter(
                x=plot_df['Date'],
                y=plot_df['Close'],
                mode='lines',
                name='History',
                line=dict(color='#00d2be', width=2)
            ))

            fig.add_trace(go.Scatter(
                x=[target_date_str],
                y=[price_pred],
                mode='markers',
                name='AI Prediction',
                marker=dict(color='red' if 'SELL' in signal else 'green', size=14, symbol='star')
            ))

            fig.add_trace(go.Scatter(
                x=[plot_df['Date'].iloc[-1], target_date_str],
                y=[plot_df['Close'].iloc[-1], price_pred],
                mode='lines',
                line=dict(color='gray', dash='dot'),
                showlegend=False
            ))

            fig.update_layout(template="plotly_dark", height=400)
            st.plotly_chart(fig, use_container_width=True)

            st.info(f"ℹ️ **Analysis:** The AI analyzed market structure up to **{decision_date_str}**. "
                    f"It detected a **{change_pct:.2f}%** potential move based on technicals "
                    "and (simulated) knowledge graph sentiment.")

else:
    st.info(" Select a date in the sidebar and click 'Run Analysis' to start.")

In [ ]:
%%writefile app.py
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
import plotly.graph_objects as go
import os

TICKER = "NVDA"
MODEL_FILE = "NVDA_Sniper_Model.pth"
SEQ_LEN = 5
DEVICE = torch.device('cpu')

class IntegratedTrader(nn.Module):
    def __init__(self, node_feat_dim, kpi_dim, gnn_out=32, lstm_hidden=128):
        super(IntegratedTrader, self).__init__()
        self.gnn1 = GCNConv(node_feat_dim, 64)
        self.gnn2 = GCNConv(64, gnn_out)
        self.lstm = nn.LSTM(gnn_out + kpi_dim, lstm_hidden, batch_first=True, dropout=0.2)
        self.head = nn.Linear(lstm_hidden, 1)

    def forward(self, graph_list, kpi_tensor):
        sentiment_vecs = []
        for g in graph_list:
            x, edge_index = g.x.to(DEVICE), g.edge_index.to(DEVICE)
            x = F.relu(self.gnn1(x, edge_index))
            x = self.gnn2(x, edge_index)
            batch = torch.zeros(x.size(0), dtype=torch.long).to(DEVICE)
            day_vec = global_mean_pool(x, batch)
            sentiment_vecs.append(day_vec)

        sentiment_seq = torch.stack(sentiment_vecs, dim=1)
        fusion = torch.cat((sentiment_seq, kpi_tensor), dim=2)
        lstm_out, _ = self.lstm(fusion)
        return self.head(lstm_out[:, -1, :])

def get_historical_stock_data(end_date_str):
    end_dt_obj = datetime.strptime(end_date_str, '%Y-%m-%d') + timedelta(days=1)
    query_end = end_dt_obj.strftime('%Y-%m-%d')
    start_dt_obj = end_dt_obj - timedelta(days=90)
    query_start = start_dt_obj.strftime('%Y-%m-%d')

    df = yf.download(TICKER, start=query_start, end=query_end, progress=False)
    if isinstance(df.columns, pd.MultiIndex): df.columns = [c[0] for c in df.columns]
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    df['SMA_14'] = df['Close'].rolling(window=14).mean()
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Return'] = df['Close'].pct_change()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    return df[df['Date'] <= end_date_str].copy()

def get_trading_day_before(target_date_obj):
    prev_dt = target_date_obj - timedelta(days=1)
    while prev_dt.weekday() > 4:
        prev_dt -= timedelta(days=1)
    return prev_dt.strftime('%Y-%m-%d')

st.set_page_config(page_title="NVDA Sniper Agent", page_icon="", layout="wide")
st.title(" NVDA Sniper: AI Investment Agent")
st.markdown("**Wanna invest in NVIDIA?** Let the AI check the **Knowledge Graph** and **Price Trends**.")

st.sidebar.header("⚙️ Configuration")
target_date_input = st.sidebar.date_input("Target Date for Prediction", datetime.today() + timedelta(days=1))
target_date_str = target_date_input.strftime('%Y-%m-%d')

if st.sidebar.button(" Run Analysis"):
    with st.spinner(f"Traveling back in time to {target_date_str}..."):
        decision_date_str = get_trading_day_before(target_date_input)
        df_seq = get_historical_stock_data(decision_date_str)

        if len(df_seq) < SEQ_LEN:
            st.error(f"Not enough data found ending on {decision_date_str}.")
        else:
            KPI_COLS = ['Close', 'RSI', 'MACD', 'SMA_14', 'Return']
            scaler = MinMaxScaler()
            df_scaled = df_seq.copy()
            df_scaled[KPI_COLS] = scaler.fit_transform(df_seq[KPI_COLS])

            seq_kpis = df_scaled[KPI_COLS].tail(SEQ_LEN).values
            kpi_tensor = torch.tensor(seq_kpis, dtype=torch.float).unsqueeze(0).to(DEVICE)

            model_loaded = False
            DETECTED_NODES = 50

            if os.path.exists(MODEL_FILE):
                try:
                    state_dict = torch.load(MODEL_FILE, map_location=DEVICE)

                    DETECTED_NODES = state_dict['gnn1.lin.weight'].shape[1]

                    model = IntegratedTrader(node_feat_dim=DETECTED_NODES, kpi_dim=len(KPI_COLS), lstm_hidden=128).to(DEVICE)
                    model.load_state_dict(state_dict)
                    model.eval()
                    model_loaded = True
                except Exception as e:
                    st.sidebar.error(f"Error loading brain: {e}")
                    model = IntegratedTrader(node_feat_dim=DETECTED_NODES, kpi_dim=len(KPI_COLS), lstm_hidden=128).to(DEVICE)
            else:
                st.warning("⚠️ Model not found. Using Random Weights.")
                model = IntegratedTrader(node_feat_dim=DETECTED_NODES, kpi_dim=len(KPI_COLS), lstm_hidden=128).to(DEVICE)

            x = torch.eye(DETECTED_NODES)
            edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
            graph_snapshot = Data(x=x, edge_index=edge_index)
            graph_seq = [graph_snapshot] * SEQ_LEN

            with torch.no_grad():
                pred_scaled = model(graph_seq, kpi_tensor).item() if model_loaded else 0.55

            dummy = np.zeros((1, len(KPI_COLS)))
            dummy[0, 0] = pred_scaled
            price_pred = scaler.inverse_transform(dummy)[0, 0]

            last_close = df_seq['Close'].iloc[-1]
            change_pct = ((price_pred - last_close) / last_close) * 100

            col1, col2, col3 = st.columns(3)
            col1.metric("Last Close", f"${last_close:.2f}", f"On {decision_date_str}")
            col2.metric("AI Target", f"${price_pred:.2f}", f"{change_pct:.2f}%")

            if price_pred > last_close * 1.005: signal, color = "BUY ", "green"
            elif price_pred < last_close * 0.995: signal, color = "SELL ", "red"
            else: signal, color = "HOLD ", "gray"
            col3.markdown(f"### Signal: :{color}[{signal}]")

            fig = go.Figure()
            plot_df = df_seq.tail(30)
            fig.add_trace(go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='lines', name='History', line=dict(color='#00d2be')))
            fig.add_trace(go.Scatter(x=[target_date_str], y=[price_pred], mode='markers', name='AI Prediction', marker=dict(color='red' if 'SELL' in signal else 'green', size=14, symbol='star')))
            fig.add_trace(go.Scatter(x=[plot_df['Date'].iloc[-1], target_date_str], y=[plot_df['Close'].iloc[-1], price_pred], mode='lines', line=dict(color='gray', dash='dot'), showlegend=False))
            fig.update_layout(template="plotly_dark", height=400)
            st.plotly_chart(fig, use_container_width=True)
else:
    st.info(" Select a date in the sidebar to start.")

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

ngrok.kill()

NGROK_TOKEN = "37AWuu4MjUuB90VQK7gLkm3ptlK_2NCrw2uygxsntWCaZXs3e"

if NGROK_TOKEN == "PASTE_YOUR_TOKEN_HERE":
    print(" STOP! You didn't paste your token. Please do step 1 & 2 above.")
else:
    ngrok.set_auth_token(NGROK_TOKEN)


    !nohup streamlit run app.py --server.port 8501 &

    try:
        public_url = ngrok.connect(8501).public_url
        print(f" Your App is Live: {public_url}")
    except Exception as e:
        print(f"Error: {e}")